In [11]:
# preparation for demo, use default settings(args) 
import argparse

from main_open_world import get_args_parser

parser = argparse.ArgumentParser('Deformable DETR training and evaluation script', parents=[get_args_parser()])

args = parser.parse_args(args = []) # args = [] : fix the error https://chaeso-coding.tistory.com/113

print(args)

Namespace(lr=0.0002, lr_backbone_names=['backbone.0'], lr_backbone=2e-05, lr_linear_proj_names=['reference_points', 'sampling_offsets'], lr_linear_proj_mult=0.1, batch_size=2, weight_decay=0.0001, epochs=51, lr_drop=35, lr_drop_epochs=None, clip_max_norm=0.1, sgd=False, with_box_refine=False, two_stage=False, masks=False, backbone='dino_resnet50', frozen_weights=None, dilation=False, position_embedding='sine', position_embedding_scale=6.283185307179586, num_feature_levels=4, enc_layers=6, dec_layers=6, dim_feedforward=1024, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, dec_n_points=4, enc_n_points=4, aux_loss=True, set_cost_class=2, set_cost_bbox=5, set_cost_giou=2, cls_loss_coef=2, bbox_loss_coef=5, giou_loss_coef=2, focal_alpha=0.25, coco_panoptic_path=None, remove_difficult=False, output_dir='', device='cuda', seed=42, resume='', start_epoch=0, eval=False, viz=False, eval_every=5, num_workers=3, cache_mode=False, PREV_INTRODUCED_CLS=0, CUR_INTRODUCED_CLS=20, unmatched_boxe

In [12]:
import torch

from pathlib import Path
from models import build_model

device = torch.device(args.device)
if args.output_dir:
        Path(args.output_dir).mkdir(parents=True, exist_ok=True)
        
model, criterion, postprocessors, exemplar_selection = build_model(args, mode = args.model_type)
model.to(device) # 모델 생성

Invalid class range: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
DINO resnet50
running with exemplar_replay_selection


DeformableDETR(
  (transformer): DeformableTransformer(
    (encoder): DeformableTransformerEncoder(
      (layers): ModuleList(
        (0): DeformableTransformerEncoderLayer(
          (self_attn): MSDeformAttn(
            (sampling_offsets): Linear(in_features=256, out_features=256, bias=True)
            (attention_weights): Linear(in_features=256, out_features=128, bias=True)
            (value_proj): Linear(in_features=256, out_features=256, bias=True)
            (output_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout1): Dropout(p=0.1, inplace=False)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (linear1): Linear(in_features=256, out_features=1024, bias=True)
          (dropout2): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=1024, out_features=256, bias=True)
          (dropout3): Dropout(p=0.1, inplace=False)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affin

In [17]:
from datasets.torchvision_datasets.open_world import OWDetection
from datasets.coco import make_coco_transforms


args.train_set = 'owod_t1_train' # data/OWOD/ImageSets/TOWOD/owod_t1_train.txt
args.test_set = 'owod_all_task_test' # data/OWOD/ImageSets/TOWOD/owod_all_task_test.txt
args.dataset = 'TOWOD' # data/OWOD/ImageSets/TOWOD


def get_datasets(args):
    print(args.dataset)

    train_set = args.train_set
    test_set = args.test_set
    dataset_train = OWDetection(args, args.data_root, image_set=args.train_set, transforms=make_coco_transforms(args.train_set), dataset = args.dataset)
    dataset_val = OWDetection(args, args.data_root, image_set=args.test_set, dataset = args.dataset, transforms=make_coco_transforms(args.test_set))

    print(args.train_set)
    print(args.test_set)
    print(dataset_train)
    print(dataset_val)

    return dataset_train, dataset_val

dataset_train, dataset_val = get_datasets(args)

TOWOD
owod_t1_train
owod_all_task_test
Dataset OWDetection
    Number of datapoints: 16551
    Root location: ./data/OWOD
    [['train'], Compose(
    Compose(
)
)]
Dataset OWDetection
    Number of datapoints: 10246
    Root location: ./data/OWOD
    [['test'], Compose(
    Compose(
)
)]
